In [ ]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import spacy
from nltk.corpus import stopwords

In [2]:
english_stopwords = stopwords.words("english")
data = pd.read_csv(r"C:\Users\alkrd\Desktop\graduation_project\the_project\preprocssed_data\cleaned_data_USA.csv")
dataFrame = data[['newsSnippet']]

In [3]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    print("lemmatizing...")
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    text_out = []
    for text in texts:
        doc = nlp(text)
        new_text = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        final = " ".join(new_text)
        text_out.append(final)
    return text_out

lemmatized_texts = lemmatization(dataFrame['newsSnippet'].astype(str).tolist())
print("lemmitized sample: ", lemmatized_texts[100])

lemmatizing...
lemmitized sample:  championship game feature accord most ranking good team result flash brilliance sloppy play defeat night


In [4]:
def gen_words(texts):
    print("removing stop words...")
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)
print("after removing stop words: ",data_words[100])

removing stop words...
after removing stop words:  ['championship', 'game', 'feature', 'accord', 'most', 'ranking', 'good', 'team', 'result', 'flash', 'brilliance', 'sloppy', 'play', 'defeat', 'night']


In [ ]:
print("vectorizing...")
id2word = corpora.Dictionary(data_words)
corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
print("corpus? ", corpus[0])

doing some shi...
corpus?  [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1)]


In [6]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=30,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto"
)

In [9]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
29     0.452485 -0.112556       1        1  7.452202
12     0.429469  0.151273       2        1  6.767148
28    -0.439290  0.132086       3        1  6.627320
8     -0.425800 -0.155178       4        1  6.568646
25     0.076286 -0.437798       5        1  5.255446
3     -0.184065 -0.377977       6        1  5.133401
24    -0.290419 -0.276262       7        1  4.622259
6     -0.038046  0.345565       8        1  4.225336
23     0.065476  0.403890       9        1  4.104120
0      0.215130 -0.321564      10        1  3.867806
5      0.304663 -0.215159      11        1  3.863171
22    -0.142607  0.372045      12        1  3.701598
10     0.273817  0.272737      13        1  3.403159
15    -0.285559  0.227299      14        1  3.078610
13     0.148441  0.297466      15        1  2.920916
18    -0.175366  0.244788      16        1  2.897555
2     -0.321612 -0.031523      17        1  2.802893
19    -0.232245  0.092060      18        1  2.681150
14     0.295912  0.005436      19        1  2.544620
7     -0.052625 -0.309907      20        1  2.499406
1      0.200397 -0.047110      21        1  2.118182
11     0.067463  0.157063      22        1  2.041686
4      0.211728  0.128362      23        1  2.016328
17    -0.192296 -0.081441      24        1  1.967236
27     0.090930 -0.217907      25        1  1.742515
16    -0.032403 -0.166839      26        1  1.642952
9     -0.075126 -0.065033      27        1  1.291520
21     0.082589 -0.069034      28        1  1.168233
26    -0.034322  0.052398      29        1  0.949268
20     0.006997  0.002819      30        1  0.045317, topic_info=       Term        Freq       Total Category  logprob  loglift
201     say  670.000000  670.000000  Default  30.0000  30.0000
504   first  520.000000  520.000000  Default  29.0000  29.0000
229    year  443.000000  443.000000  Default  28.0000  28.0000
55    night  352.000000  352.000000  Default  27.0000  27.0000
410  season  356.000000  356.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
23     name    0.002745  121.916658  Topic30  -8.9816  -3.0022
24     same    0.002745   30.055884  Topic30  -8.9816  -1.6019
26     seem    0.002745   50.568415  Topic30  -8.9816  -2.1222
28     tool    0.002745    6.731690  Topic30  -8.9816  -0.1057
29    cable    0.002745    7.716916  Topic30  -8.9816  -0.2423

[1135 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
75       10  0.984771    about
610       8  0.993535   accord
413      25  0.964109  account
745      12  0.955798   accuse
169      10  0.961941      act
...     ...       ...      ...
412      24  0.983698    world
958       9  0.630445    worry
886      11  0.987559    write
150      21  0.936955   writer
229       7  0.997698     year

[615 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[30, 13, 29, 9, 26, 4, 25, 7, 24, 1, 6, 23, 11, 16, 14, 19, 3, 20, 15, 8, 2, 12, 5, 18, 28, 17, 10, 22, 27, 21])